<h1>Hola muchachos y benvenido en el RECORDO LINKAGGIO<h1>
<h3>Per prima cosa dobbiamo fare il setup dell'ambiente<h3>
<h4>Nulla di complicato e se hai giá installato la libreria recordlinkage nel tuo ambiente python puoi saltare questo passaggio<h4>
<h5>Ma considera che ha anche queste dipendenze:
    numpy,
    pandas,
    scipy,
    sklearn,
    jellyfish,
    joblib,
Inoltre raccomandate sono:
    numexpr - accelerating certain numerical operations,
    bottleneck - accelerating certain types of nan evaluations<h5>


In [ ]:
!pip install recordlinkage

<h2>Da qui inizia l'uso e costumo della libreria<h2>

In [1]:
import recordlinkage
from recordlinkage import Compare
from recordlinkage.preprocessing import clean
import pandas as pd

In [11]:
#ora proviamo a scrivere qualche esempio con un piccolo subset
df_prova = pd.read_csv(r'../geopy/subset_with_coordinates.csv')
print(df_prova.head())

   address                  category  ...   latitude  longitude
0      NaN  IT Services & Consulting  ...  19.090153  72.863713
1      NaN  IT Services & Consulting  ...  53.349379  -6.260559
2      NaN  IT Services & Consulting  ...        NaN        NaN
3      NaN                   Banking  ...  19.090153  72.863713
4      NaN                   Banking  ...  19.090153  72.863713

[5 rows x 23 columns]


<h4>Ora scriviamo un block di esempio, con esso in pratica confrontiamo solo record che hanno lo stesso valore messo nel block<h4>

In [20]:
#per esempio qui prendimo il nome che potrebbe avere senso se sono tutti proprio uguali (palese no)
indexer = recordlinkage.Index()
#nel caso di un unico df
indexer.block('fundation_year')
#nel caso di due df
#indexer.block(left_on='name', right_on='name')
pairs = indexer.index(df_prova)
print(len(pairs))

306


<h4>Ora dobbiamo comparare!<h4>
<h7>Per capirci: 
method specifica come confronto i valori, nel nostro caso jarowinkler é un approccio lazy cioé anche se non sono perfettamente identici ma molto simili fanno match
threshold é la soglia oltre la quale consideriamo due record come simili e funziona solo con metodi lazy
label é solo il nome che vogliamo dare al confronti
<h7>

In [21]:
compare = Compare()

compare.string('address', 'address', method='jarowinkler', threshold=0.85, label='address')
compare.string('city', 'city', method='jarowinkler', threshold=0.85, label='city')
compare.string('country', 'country', method='jarowinkler', threshold=0.85, label='country')

features = compare.compute(pairs, df_prova)
print(features.head())

# Consideriamo un record un match se almeno 2 confronti su 3 danno un punteggio alto
matches = features[features.sum(axis=1) >= 1]

print(matches)

        address  city  country
53  0       0.0   1.0      0.0
102 0       0.0   0.0      0.0
    53      0.0   0.0      0.0
31  1       0.0   0.0      0.0
111 1       0.0   0.0      0.0
         address  city  country
53  0        0.0   1.0      0.0
4   3        0.0   1.0      0.0
154 3        0.0   1.0      0.0
    4        0.0   1.0      0.0
145 25       0.0   1.0      0.0
153 5        0.0   1.0      0.0
159 29       0.0   1.0      0.0
80  43       0.0   1.0      0.0
170 147      0.0   1.0      0.0
194 82       0.0   1.0      0.0
96  27       0.0   1.0      0.0
142 36       0.0   1.0      0.0
174 24       0.0   1.0      0.0
126 26       0.0   1.0      0.0
59  28       0.0   1.0      0.0
103 28       0.0   1.0      0.0
    59       0.0   1.0      0.0
52  51       0.0   1.0      0.0
95  51       0.0   1.0      0.0
    52       0.0   1.0      0.0
115 51       0.0   1.0      0.0
    52       0.0   1.0      0.0
    95       0.0   1.0      0.0
141 91       0.0   1.0      0.0
155 91       0